In [1]:
import qutip as qt
import math
from libraries import lib
import numpy as np

In [2]:
s00 = qt.basis(4, 0)
s00

Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]]

In [3]:
qt.bell_state()

Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]]

In [4]:
xplus = 1/math.sqrt(2) * (qt.basis(2, 0) + qt.basis(2, 1))
xminus = 1/math.sqrt(2) * (qt.basis(2, 0) - qt.basis(2, 1))

In [5]:
s00.transform([qt.tensor([xplus, xplus]), qt.tensor([xplus, xminus]), qt.tensor([xminus, xplus]), qt.tensor([xminus, xminus])])

Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.5]
 [0.5]
 [0.5]
 [0.5]]

In [6]:
s00[0][0]

np.complex128(1+0j)

In [7]:
def absolute_z_magnetization(N, psi):
    total_mag = 0
    dim = 2 ** N
    for basis_vec in range(dim):
        down_spins = lib.count_magnetization(basis_vec)
        total_mag += abs(abs(psi[basis_vec]) ** 2 * ((N - down_spins) - down_spins))
    return total_mag

def signed_z_magnetization(N, psi):
    total_mag = 0
    dim = 2 ** N
    for basis_vec in range(dim):
        down_spins = lib.count_magnetization(basis_vec)
        total_mag += abs(psi[basis_vec]) ** 2 * ((N - down_spins) - down_spins)
    return total_mag

def z_expect(N, psi): # THIS IS WRONG
    op = qt.tensor([qt.sigmaz()] * N)
    op.dims = [2 ** N, 2 ** N]
    return psi.dag() * op * psi# psi.dag().full() @ qt.tensor([qt.sigmax()] * N).full() @ psi.full()

In [8]:
N = 2
random = qt.rand_ket(2 ** N)
print(random)
print(signed_z_magnetization(N, random))
print(z_expect(N, random))

Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.57777759+0.17879549j]
 [-0.50298006+0.16461559j]
 [-0.51884306-0.25669347j]
 [-0.13744837+0.01167426j]]
[0.69353285]
(-0.23035377562873827+0j)


In [9]:
for n in range(0, 4):
    print(qt.basis(4, n).dag().full() @ (qt.tensor([qt.sigmaz(), qt.qeye(2)]) + qt.tensor([qt.qeye(2), qt.sigmaz()])).full() @ qt.basis(4, n).full())

[[2.+0.j]]
[[0.+0.j]]
[[0.+0.j]]
[[-2.+0.j]]


In [10]:
for n in range(0, 4):
    print(signed_z_magnetization(2, qt.basis(4, n)) )

[2.]
[0.]
[0.]
[-2.]


In [11]:
for n in range(0, 4):
    print(absolute_z_magnetization(2, qt.basis(4, n)))

[2.]
[0.]
[0.]
[2.]


In [12]:
op = qt.Qobj([[1, 0], [0, 0]])
op

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [13]:
for n in range(0, 4):
    print(qt.basis(4, n).dag().full() @ (qt.tensor([op, qt.qeye(2)]) + qt.tensor([qt.qeye(2), op])).full() @ qt.basis(4, n).full())

[[2.+0.j]]
[[1.+0.j]]
[[1.+0.j]]
[[0.+0.j]]


In [14]:
# check that for x - expect, transform into x basis and then count magnetization is the same as using operator
def z_to_x(N, psi):
    dim = 2 ** N
    basis = []
    for i in range(dim):
        basis.append(qt.tensor([[xplus, xminus][(i >> (N - k - 1)) & 1] for k in range(0, N)]))
    return psi.transform(basis)

In [15]:
z_to_x(3, qt.basis(8, 4))

Quantum object: dims=[[8], [1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.35355339]
 [ 0.35355339]
 [ 0.35355339]
 [ 0.35355339]
 [-0.35355339]
 [-0.35355339]
 [-0.35355339]
 [-0.35355339]]

In [16]:
qt.basis(4, 2).transform([qt.tensor([xplus, xplus]), qt.tensor([xplus, xminus]), qt.tensor([xminus, xplus]), qt.tensor([xminus, xminus])])

Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.5]
 [ 0.5]
 [-0.5]
 [-0.5]]

In [30]:
def calc_x_1(N, psi):
    x_psi = z_to_x(N, psi)
    return lib.z_magnetization(N, x_psi, signed = True)
def calc_x_2(N, psi):
    sxis = []
    for i in range(N):
        op_list = [qt.qeye(2)] * N
        op_list[i] = qt.sigmax()
        sxis.append(qt.tensor(op_list))
    final_op = sum(sxis)
    final_op.dims = [2 ** N, 2 ** N]
    return (psi.dag() * final_op * psi).real

In [31]:
N = 5
random = qt.rand_ket(2 ** N)
print(random)
print(calc_x_1(N, random))
print(calc_x_2(N, random))

Quantum object: dims=[[np.int64(32)], [1]], shape=(32, 1), type='ket', dtype=Dense
Qobj data =
[[-0.15241334-0.20150079j]
 [ 0.03088322-0.19394474j]
 [-0.00744368+0.00835743j]
 [-0.13948192+0.14968131j]
 [-0.03782846+0.02514129j]
 [ 0.00842107-0.18348726j]
 [ 0.01795321+0.16603519j]
 [ 0.06932171+0.15413838j]
 [ 0.1316942 +0.19106579j]
 [-0.01541386-0.13246297j]
 [ 0.17973153-0.0140569j ]
 [-0.05701454-0.16364619j]
 [ 0.2842929 +0.05217469j]
 [ 0.07119234+0.03282612j]
 [-0.01224094+0.01080121j]
 [ 0.09490266+0.12045535j]
 [-0.03053226+0.19614492j]
 [-0.21205603+0.03413053j]
 [ 0.01891619-0.31470876j]
 [-0.07236792-0.02881111j]
 [-0.18320877-0.07531551j]
 [ 0.18385117-0.09613738j]
 [-0.15785521+0.00354079j]
 [ 0.10961594+0.05975522j]
 [-0.02313002-0.2140796j ]
 [ 0.02542404+0.00068535j]
 [ 0.15179367+0.04407655j]
 [ 0.07242617+0.13460161j]
 [ 0.11785194+0.12656268j]
 [-0.03780954+0.06693053j]
 [-0.13417914-0.12836565j]
 [ 0.17894623-0.06510113j]]
[-0.41227393]
-0.4122739265185646


In [ ]:
# did not test absolute operator method here